# Agent-based peptide discovery (coding)

In [1]:
import autogen
import os
import tqdm
import time
import openai
from openai import AsyncOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from IPython.display import Markdown, display
os.environ["AUTOGEN_USE_DOCKER"] = "0"
from tqdm import tqdm


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

/opt/anaconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
###################### HF
HF_token= ''
token = HF_token
from huggingface_hub import login
login(token=token)

######### OPENAI ###########
os.environ['OPENAI_API_KEY'] = "" # add your api

/opt/anaconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Using GPT-4 - Main setup starts here

In [3]:
device = "cpu"
import transformers
autogen.__version__, openai.__version__, transformers.__version__ #make sure to update both autogen and openai to their newsest versions 
# ('0.2.2', '1.6.0', '4.35.1')

('0.9.10', '1.109.1', '4.44.2')

In [4]:
from llm_config import config_list
import agent_functions as func
import agents
import autogen

In [ ]:
def _reset_agents():
    agents.user_proxy.reset()
    agents.code_planner.reset()
    agents.code_assistant.reset()
    agents.ml_coder.reset()

_reset_agents()

manager_llm_config = {
    "config_list": config_list,
    "seed": 45,
}

groupchat = autogen.GroupChat(
    agents=[agents.user_proxy, agents.code_planner, agents.code_assistant, agents.ml_coder, 
               ], messages=[], max_round=150, 
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config = manager_llm_config, 
                system_message='This agent repeats the following steps: Dynamically selecting a speaker, collecting response, and \
broadcasting the message to the group.')

In [6]:
# agents.user_proxy.initiate_chat(
#     recipient=manager,
#     message='''
# Can you provide some names of antimicrobial resistant bacteria?
# '''
# )
agents.user_proxy.initiate_chat(
    recipient=manager,
    message='''
Given local files: positive_data_fasta_path = 'train_fungi_AMP.fasta' containing antifungal peptide sequences and
negative_data_fasta_path = 'train_negative_peptide.fasta' containing non-antifungal peptide sequences, write codes that can train a deep learing model to classify antifungal peptides vs. non-antifungal peptides. Finetune ESM2 model for this task. Provide the complete code with necessary imports.
'''
)


user_proxy (to chat_manager):


Given local files: positive_data_fasta_path = 'train_fungi_AMP.fasta' containing antifungal peptide sequences and
negative_data_fasta_path = 'train_negative_peptide.fasta' containing non-antifungal peptide sequences, write codes that can train a deep learing model to classify antifungal peptides vs. non-antifungal peptides. Finetune ESM2 model for this task. Provide the complete code with necessary imports.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Your aim is to train a deep learning model using positive and negative data you already possess and fine-tune the existing ESM2 model for this task. It includes classes labeled as antifungal peptides and non-antifungal peptides. This falls squarely within the realm of binary classification.

Here's an initial plan:

Step 1. Load and preprocess your dataset from FASTA files to prepare it for the model. This includes con

ChatResult(chat_id=298481404645565881608201006960904020788, chat_history=[{'content': "\nGiven local files: positive_data_fasta_path = 'train_fungi_AMP.fasta' containing antifungal peptide sequences and\nnegative_data_fasta_path = 'train_negative_peptide.fasta' containing non-antifungal peptide sequences, write codes that can train a deep learing model to classify antifungal peptides vs. non-antifungal peptides. Finetune ESM2 model for this task. Provide the complete code with necessary imports.\n", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Your aim is to train a deep learning model using positive and negative data you already possess and fine-tune the existing ESM2 model for this task. It includes classes labeled as antifungal peptides and non-antifungal peptides. This falls squarely within the realm of binary classification.\n\nHere's an initial plan:\n\nStep 1. Load and preprocess your dataset from FASTA files to prepare it for the model. This includes converting pept

In [7]:
agents.user_proxy.send(
    recipient=manager,
    message='''
Given model trained for AMP classification, write codes for the function that can load the model file and inference this model to classify antifungal peptides vs. non-antifungal peptides. Provide the complete code with necessary imports.
'''
)

user_proxy (to chat_manager):


Given model trained for AMP classification, write codes for the function that can load the model file and inference this model to classify antifungal peptides vs. non-antifungal peptides. Provide the complete code with necessary imports.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

***** Suggested function call: ML_Coder *****
Arguments: 
The user wants a function for loading the trained model and using it to classify peptides. The function should take as input parameters the file path of a saved model (model_path) and the sequence to classify (sequence). 

As output, the function should issue the class prediction (0 for non-antifungal peptide, 1 for antifungal peptide) and the prediction probabilities for each class. 

Can you provide the Python code for this function, using PyTorch for model loading and inference and BioPython to assist with handling the sequence?